# Kernel ridge regression for functions in the RKHS

This notebook is for least-squares KRR for synthetic data WITHOUT cross-validation.

Question: given $f\in \mathcal{H}$, does KRR+KT outperform KRR+ST for fixed bandwidth and regularization parameters?

Use this notebook to run one of three types of experiments:
1. **Kernel experiment** (```varying_variable='kernel'```): 
    
    Vary the kernel, keeping the distrubution of X and the function f fixed

2. **Dimension experiment** (```varying_variable='d'```): 

    Vary the dimension of X, keeping the function f and kernel fixed (X='gauss')

3. **Mixtures experiment** (```varying_variable='M'```): 

    Vary the number of mixtures in X, keeping the function f and kernel fixed (X='mog', d=2)

In [1]:
# install using `conda install -c conda-forge line_profiler`
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_is_fitted
import numpy as np
from numpy.linalg import LinAlgError
# set global seed
# np.random.seed(123)

import pandas as pd
from datetime import datetime
from copy import deepcopy
# utils for plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


# utils for timing
from goodpoints.tictoc import tic, toc, TicToc
# utils for kernel ridge regression
from goodpoints.krr.util_estimators import get_estimator, get_sigma_heuristic
# utils for evaluating kernels
from goodpoints.krr.thin.util_k_mmd import kernel_eval, to_regression_kernel
# utils for generate samples from the data distribution
from goodpoints.krr.util_sample import get_Xy, ToyData , get_toy_dataset #, get_housing_dataset
# utils for dataset thinning
from goodpoints.krr.thin.util_thin import sd_thin, kt_thin2
# # falkon baseline
# from goodpoints.krr.falkon.util_falkon_estimators import KernelRidgeFalkon

In [3]:
# add this to be able to render plotly plots in non-vscode notebooks
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [4]:
# https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express
import plotly.colors as colors

# fig = px.colors.qualitative.swatches()
# fig.show()

## Set hyperparameters

We use the following model to create the dataset for $i=1,\ldots,n$:
$$y_i=f(X_i)+w_i,\quad X_i\sim p_{data}(x)$$
where $w_i\sim \mathcal{N}(0,\sigma^2)$ i.i.d.

In [35]:
### Toy dataset parameters

n = 2**10

X_name = 'unif'  # ['gauss', 'mog']
X_var = 1
f_name = 'sum-laplace' # ['sin', 'stair', 'quad', 'sum-gauss', 'sum-laplace', 'step']
noise = 0.1

d = 1
M = 2
k = 8 # number of anchor points for sum-gauss and sum-laplace

### Regression parameters

kernel = 'laplace'  # ['gauss', 'laplace']
sigma =  0.25
alpha = 1

### Experiment parameters

varying_variable = 'kernel' # ['kernel', 'd', 'M']
n_repeats = 100
save = False
logn_lo = 8
logn_hi = 15
use_cross_validation = False
n_jobs = 1

### Thinning parameters

m = None # Thinned dataset will have size n/2**m

In [36]:
# parameter checks

assert X_name in ['unif', 'gauss', 'mog']
if X_name == 'mog': assert d <= 2

assert f_name in ['sin', 'stair', 'quad', 'sum-gauss', 'sum-laplace', 'step']

assert kernel in ['gauss', 'laplace']

assert varying_variable in ['kernel', 'd', 'M']
if X_name == 'mog': assert varying_variable != 'd', \
        ValueError('cannot running d-varying experiment when X is mog')
if varying_variable in ['d', 'M']: assert kernel == 'laplacian', 'laplacian is not set as default kernel'

In [37]:
# Determine auxiliary parameters
task = 'regression'
refit = 'neg_mean_squared_error'
postprocess = None

filename = '_'.join([X_name, f_name, f'k={k}', f'alpha={alpha}', f'sigma={sigma}'])
print(filename)

baseline_loss = noise**2

unif_sum-laplace_k=8_alpha=1_sigma=0.125


Kernels:
- RBF:
$$\mathbf{k}(x, y) = \exp(-\gamma ||x-y||_2^2)$$
- Laplacian:
$$\mathbf{k}(x, y) = \exp(-\gamma ||x-y||_2)$$

Median heuristic to choose the bandwidth parameter, i.e., median of squared pairwise distances:
- For Gaussian data, we can compute this exactly. Assume $X\sim \mathcal{N}(0,\sigma^2 I_d)$. For the RBF kernel, $X_1-X_2\sim \mathcal{N}(0,2\sigma^2 I_d)$. Then $(X_1-X_2)^2$ follows a chi-squared distribution with $d$ degrees of freedom, mean $d\cdot \sqrt{2}\sigma$ and median roughly $d(1-\frac{2}{9d})^3 \cdot \sqrt{2}\sigma$. For the Laplacian kernel, $||x-y||_1$ follows a folded normal distribution (https://en.wikipedia.org/wiki/Folded_normal_distribution) with median roughly $\sqrt{2}\sigma$.

Available kernels in sklearn: 
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise

## Get dataset

In [38]:
toy_data_noise = ToyData(X_name, f_name, X_var=X_var, noise=noise, 
                    d=d, M=M, k=k)
# toy_data_no_noise = ToyData(X_name, f_name, X_var=X_var, noise=noise, 
#                     d=d, M=M, k=k)
X_train, y_train = toy_data_noise.sample(n)
X_test, y_test = toy_data_noise.sample(10000)
# validation set used for cross validation
X_val, y_val = toy_data_noise.sample(10000)

print(X_train.shape, y_train.shape)

(1024, 1) (1024,)


In [39]:
np.std(X_train), np.std(y_train)

(1.0009770398031608, 0.6402002435524057)

In [40]:
def trace_X(X):
    d = X.shape[-1]

    if d== 1:
        return go.Histogram(
            x=X.squeeze(),
            histnorm='probability density',
            name='histogram (normalized)',
            nbinsx=200,
        )

    elif d==2:
        x,y = X[:,0], X[:,1]

        return go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=dict(
                symbol='x',
                opacity=0.5,
                color='white',
                size=8,
                line=dict(width=1),
            ),
            name='X data points'
        )

    print(f"cannot plot X with dimension {d}")

def contour(X, pdf_eval):
    d = X.shape[-1]

    if d== 1:
        ticks = 100
        X_coor = np.linspace(X.min(), X.max(), ticks)
        pdf = pdf_eval(X_coor)
        return go.Scatter(
            x=X_coor, 
            y=pdf.squeeze(), 
            mode='lines',
            name='pdf'
        )

    elif d==2:
        x,y = X[:,0], X[:,1]

        ticks = 100
        X_coor, Y_coor = np.linspace(x.min(), x.max(), ticks), np.linspace(y.min(), y.max(), ticks)
        XY_grid = np.stack(np.meshgrid(X_coor, Y_coor), axis=-1)

        pdf = pdf_eval(XY_grid)
        # print(pdf)

        return go.Contour(
            x=X_coor, 
            y=Y_coor, 
            z=pdf.squeeze(), 
            showscale=False, 
            contours=dict(coloring='none'),
            name='pdf'
        )
    
    print(f'cannot plot contour in {d} dimensions')

In [41]:
fig = go.Figure(
    data=[
        trace_X(X_train), 
        # contour(X_train, toy_data_noise.pdf)
    ], 
    # layout=dict(showlegend=True)
)
fig.show()

In [42]:
def trace_Xy(X, y, name=None, color=None, alpha=0.5):
    d = X.shape[-1]

    if d==1:
        return go.Scatter(
            x=X.squeeze(), 
            y=y, 
            mode='markers',
            name=name,
            opacity=alpha,
            marker=dict(
                color=color,
            )
        )

    elif d==2:
        x1,x2 = X[:,0], X[:,1]
        return go.Scatter3d(x=x1, y=x2, z=y, mode='markers', name=name, marker=dict(
            symbol='circle',
            opacity=alpha,
            color=color,
            size=2,
            # line=dict(width=1),
        ))

    else:
        print(f"cannot plot data with dimension {d}")

In [45]:
def hnorm(k):
    from goodpoints.krr.thin.util_k_mmd import laplacian, gauss
    anchor_points = np.linspace(-1, 1, k)[:, np.newaxis]
    # print(anchor_points)

    if f_name == 'sum-gauss':
        K = gauss(anchor_points, anchor_points, 0.25)
    elif f_name == 'sum-laplace':
        K = laplacian(anchor_points, anchor_points, 0.25)
    else:
        raise ValueError(f'f_name {f_name} not supported')
    # print(K)
    
    return np.sum(K)
    # return 1/(k**2) * np.sum(K)

In [67]:
hnorm(2)

2.000670925255805

In [52]:
fig = go.Figure(data=[
    trace_Xy(X_train, y_train, name='train'),
    # trace_Xy(X_test, y_test, name='test'),
])
fig.update_layout(
    title=f'X={X_name}, f={f_name}, std[f]={np.std(y_train):.4f}, hnorm[f]={hnorm(k):.4f}',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_function.png")

### Thinning functions

In [53]:
%%time
sd_coreset = sd_thin(X_train, m=m)
print('sd coreset:', len(sd_coreset))
X_train_sd_thin, y_train_sd_thin = X_train[sd_coreset], y_train[sd_coreset]

sd coreset: 32
CPU times: user 418 µs, sys: 136 µs, total: 554 µs
Wall time: 496 µs


In [54]:
from functools import partial

# KERNEL THINNING

# Define kernel params
params_k_swap = {"name": kernel, "var": sigma**2, "d": int(d)}
params_k_split = {"name": kernel, "var": sigma**2, "d": int(d)}

split_kernel = partial(kernel_eval, params_k=params_k_split)
swap_kernel = partial(kernel_eval, params_k=params_k_swap)

regression_split_kernel = to_regression_kernel(split_kernel)
regression_swap_kernel = to_regression_kernel(swap_kernel)

In [55]:
# from goodpoints import compress
# %lprun -f kt_thin2 
kt_coreset = kt_thin2(X_train, split_kernel, swap_kernel, m=m)

In [56]:

print('kt coreset:', len(kt_coreset))
X_train_kt_thin, y_train_kt_thin = X_train[kt_coreset], y_train[kt_coreset]

kt coreset: 32


In [57]:
Xy_train = get_Xy(X_train, y_train)
print(Xy_train.shape)

(1024, 2)


In [58]:
# %lprun -f kt_thin2 
ktr_coreset = kt_thin2(Xy_train, regression_split_kernel, regression_swap_kernel, m=m)

In [59]:

X_train_ktr_thin, y_train_ktr_thin = X_train[ktr_coreset], y_train[ktr_coreset]

In [60]:
# fig = make_subplots(
#     rows=4, cols=1, 
#     subplot_titles=['full', 'st' ,'kt (non-regression)', 'kt (regression)'],
#     shared_xaxes=True,
# )

# fig.add_trace(
#     trace_X(X_train),
#     row=1, col=1,
# )
# fig.add_trace(
#     contour(X_train, toy_data_noise.pdf),
#     row=1, col=1,
# )
# fig.add_trace(
#     trace_X(X_train_sd_thin),
#     row=2, col=1
# )
# fig.add_trace(
#     contour(X_train_sd_thin, toy_data_noise.pdf),
#     row=2, col=1
# )
# fig.add_trace(
#     trace_X(X_train_kt_thin),
#     row=3, col=1
# )
# fig.add_trace(
#     contour(X_train_kt_thin, toy_data_noise.pdf),
#     row=3, col=1
# )
# fig.add_trace(
#     trace_X(X_train_ktr_thin),
#     row=4, col=1
# )
# fig.add_trace(
#     contour(X_train_ktr_thin, toy_data_noise.pdf),
#     row=4, col=1
# )
# fig.update_layout(height=900, showlegend=False)
# fig.show()

## KRR (Full)

In [61]:
krr_full = get_estimator(
    task, 
    'full', 
    alpha=alpha, 
    kernel=kernel, 
    sigma=sigma, 
)

In [62]:
krr_full.get_params()

{'M': None,
 'alpha': 1,
 'kernel': 'laplace',
 'postprocess': None,
 'sigma': 0.125}

In [63]:
%%time
krr_full.fit(X_train, y_train)

CPU times: user 28.8 ms, sys: 21.2 ms, total: 50.1 ms
Wall time: 26.1 ms


array([[1.00000000e+00, 7.83638283e-01, 9.99825829e-05, ...,
        3.47670461e-01, 5.21919947e-03, 1.27598783e-02],
       [7.83638283e-01, 1.00000000e+00, 1.27587670e-04, ...,
        4.43661914e-01, 6.66021502e-03, 1.62828674e-02],
       [9.99825829e-05, 1.27587670e-04, 1.00000000e+00, ...,
        2.87578596e-04, 1.91566894e-02, 7.83570037e-03],
       ...,
       [3.47670461e-01, 4.43661914e-01, 2.87578596e-04, ...,
        1.00000000e+00, 1.50119152e-02, 3.67010710e-02],
       [5.21919947e-03, 6.66021502e-03, 1.91566894e-02, ...,
        1.50119152e-02, 1.00000000e+00, 4.09032073e-01],
       [1.27598783e-02, 1.62828674e-02, 7.83570037e-03, ...,
        3.67010710e-02, 4.09032073e-01, 1.00000000e+00]])

In [64]:
%%time
print('Score:', krr_full.score(X_test, y_test))

Score: 0.9735687559946229
CPU times: user 168 ms, sys: 75.2 ms, total: 243 ms
Wall time: 165 ms


In [65]:
print('MSE:', mean_squared_error(y_test, krr_full.predict(X_test)))

MSE: 0.010829661567248005


In [66]:
fig = go.Figure(data=[
    # trace_Xy(X_train, y_train, name='train', alpha=1),
    trace_Xy(X_test, y_test, name='test', alpha=0.1, color='red'),
    trace_Xy(X_test, krr_full.predict(X_test), name='pred', alpha=0.4, color=colors.qualitative.Plotly[2]),
])
fig.update_layout(
    title='full',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_full.png")

## Standard Thinning (ST)

In [26]:
krr_sd_thin = get_estimator(
    task, 
    'st', 
    alpha=alpha / np.power(n, 1/4), 
    kernel=kernel, 
    sigma=sigma, 
    m=m, 
)

In [27]:
krr_sd_thin.get_params()

{'alpha': 0.17677669529663687,
 'kernel': 'laplace',
 'm': None,
 'postprocess': None,
 'sigma': 0.125}

In [28]:
%%time
krr_sd_thin.fit(X_train, y_train)

CPU times: user 884 µs, sys: 1.48 ms, total: 2.37 ms
Wall time: 3.29 ms


In [29]:
%%time
print('Score:', krr_sd_thin.score(X_test, y_test))

Score: 0.24631835891191078
CPU times: user 37.9 ms, sys: 30.7 ms, total: 68.5 ms
Wall time: 11.1 ms


In [30]:
print('MSE:', mean_squared_error(y_test, krr_sd_thin.predict(X_test)))

MSE: 0.03088846766721582


In [54]:
fig = go.Figure(data=[
    trace_Xy(krr_sd_thin.X_fit_, krr_sd_thin.y_fit_, name='train coreset', alpha=1),
    trace_Xy(X_test, y_test, name='test', alpha=0.1),
    trace_Xy(X_test, krr_sd_thin.predict(X_test), name='pred', alpha=0.4)
])
fig.update_layout(
    title='st',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_st.png")

## Kernel Thinning (KT)

In [32]:
krr_kt_thin = get_estimator(
    task,
    'kt', 
    kernel=kernel, 
    alpha=alpha / np.power(n, 1/4),
    sigma=sigma, 
    m=m, 
)

In [33]:
krr_kt_thin

KernelRidgeKTRegressor(alpha=0.17677669529663687, sigma=0.125)

In [34]:
krr_kt_thin.get_params()

{'alpha': 0.17677669529663687,
 'kernel': 'laplace',
 'm': None,
 'postprocess': None,
 'sigma': 0.125,
 'store_K': True,
 'ydim': 1}

In [35]:
# %%time
# To run line profiler, uncomment the next line
# %lprun -f krr_kt_thin.fit krr_kt_thin.fit(X_train, y_train)
krr_kt_thin.fit(X_train, y_train)

In [36]:
%%time
print('Score:', krr_kt_thin.score(X_test, y_test))

Score: 0.6658119070079105
CPU times: user 35.3 ms, sys: 26.1 ms, total: 61.5 ms
Wall time: 11.9 ms


In [37]:
print('MSE:', mean_squared_error(y_test, krr_kt_thin.predict(X_test)))

MSE: 0.013696178256713279


In [38]:
fig = go.Figure(data=[
    trace_Xy(krr_kt_thin.X_fit_, krr_kt_thin.y_fit_, name='train coreset',alpha=1),
    trace_Xy(X_test, y_test, name='test', alpha=0.1),
    trace_Xy(X_test, krr_kt_thin.predict(X_test), name='pred', alpha=0.2)
])
fig.update_layout(
    title='kt',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_kt.png")

## FALKON

In [39]:
krr_falkon = get_estimator(
    task,
    'falkon',
    kernel=kernel,
    sigma=sigma,
    # alpha=alpha,
    alpha=1e-4, # falkon works best with very low alpha
    m=m,
)

No module named 'falkon'


In [40]:
if krr_falkon: krr_falkon.get_params()

In [41]:
if krr_falkon:
    # %lprun -f krr_falkon.fit 
    krr_falkon.fit(X_train, y_train)

In [42]:
%%time
if krr_falkon:
    print('Score:', krr_falkon.score(X_test, y_test))
    print('MSE:', mean_squared_error(y_test, krr_falkon.predict(X_test)))
    # print(np.mean((y_test - krr_falkon.predict(X_test).squeeze())**2))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.1 µs


In [43]:
if krr_falkon:
    fig = go.Figure(data=[
        trace_Xy(X_test, y_test, name='test', alpha=0.1),
        trace_Xy(X_test, krr_falkon.predict(X_test).squeeze(), name='pred', alpha=0.2)
    ])
    fig.update_layout(
        title='falkon',
        height=400,
        width=800,
    )
    fig.show()
    # save fig
    if save: fig.write_image(f"figures/{filename}_falkon.png")

## FALKON + KT

In [44]:
# krr_falkon_kt = get_estimator(
#     task,
#     'falkon+kt',
#     kernel=kernel,
#     sigma=sigma,
#     alpha=alpha,
#     m=m,
# )

In [45]:
# %lprun -f krr_falkon_kt.fit krr_falkon_kt.fit(X_train, y_train)

In [46]:
# %%time
# if krr_falkon_kt:
#     print('Score:', krr_falkon_kt.score(X_test, y_test))
#     print('RMSE:', np.sqrt(mean_squared_error(y_test, krr_falkon_kt.predict(X_test))))

## Grid Search

We now run a full grid search with cross validation across different-size datasets.

Reference: https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_hist_grad_boosting_comparison.html#sphx-glr-auto-examples-ensemble-plot-forest-hist-grad-boosting-comparison-py

In [47]:
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.metrics import r2_score, accuracy_score

In [48]:
"""
Varying variables (during grid search, these are NOT parallelized)
"""

if varying_variable == 'kernel':
    varying_variable_values = ['gauss', 'laplace',]
elif varying_variable == 'd':
    varying_variable_values = [2,4, 10, 100]
elif varying_variable == 'M':
    varying_variable_values = [3, 4, 6, 8]
else:
    raise ValueError(f"invalid experiment: cannot vary '{varying_variable}'")

In [49]:
print('Running experiment with varying variable:', varying_variable)
print('taking values:', varying_variable_values)

Running experiment with varying variable: kernel
taking values: ['gauss', 'laplace']


In [50]:
# # Default param grid to search for each model
default_param_grid = {
    "sigma" :   [sigma,],
    "alpha" :   [1e1, 1e-0, 1e-1, 1e-2, 1e-3, 1e-4],

    # "sigma" :   1/np.sqrt(2*np.array([0.5, 1., 2, 5.])),
    # "alpha" :   [0.01, 0.02]
}
falkon_param_grid = {
    "sigma" :   [sigma,],
    "alpha" :   [1e-4, 1e-5,1e-6, 1e-7, 0],
}


# Model constructors and data size for each model
# We allow for different data sizes to avoid running Full KR on large datasets
model_configs = {
    'full' : {
        'logn' : np.arange(logn_lo, logn_hi, 2),
        # 'logn' : np.arange(8, 13, 2),
        'kwargs': {
            'postprocess' : postprocess,
            # 'alpha' : alpha,
            # 'sigma' : sigma,
        },
        'param_grid' : default_param_grid,
    },
    'st' : {
        'logn' : np.arange(logn_lo, logn_hi, 2),
        # 'logn' : np.arange(8, 13, 2),
        'kwargs' : {
            'm' : m,
            'postprocess' : postprocess,
            # 'alpha' : alpha,
            # 'sigma' : sigma,
        },
        'param_grid' : default_param_grid,
    },
    'kt' : {
        'logn' : np.arange(logn_lo, logn_hi, 2),
        # 'logn' : np.arange(8, 13, 2),
        'kwargs' : {
            'm' : m,
            'postprocess' : postprocess,
            # 'alpha' : alpha,
            # 'sigma' : sigma,
        },
        'param_grid' : default_param_grid,
    },
    'falkon' : {
        'logn' : np.arange(logn_lo, logn_hi, 2),
        # 'logn' : np.arange(8, 13, 2),
        'kwargs' : {
            'm' : m,
            'postprocess' : postprocess,
            # 'alpha' : alpha /100, # https://falkonml.github.io/falkon/examples/falkon_regression_tutorial.html
            # 'sigma' : sigma,
        },
        'param_grid' : falkon_param_grid,
    },
}

# cv = RepeatedKFold(n_repeats=n_repeats, n_splits=k_fold)

In [51]:
model_configs

{'full': {'logn': array([ 8, 10, 12, 14]), 'kwargs': {'postprocess': None}},
 'st': {'logn': array([ 8, 10, 12, 14]),
  'kwargs': {'m': None, 'postprocess': None}},
 'kt': {'logn': array([ 8, 10, 12, 14]),
  'kwargs': {'m': None, 'postprocess': None}},
 'falkon': {'logn': array([ 8, 10, 12, 14]),
  'kwargs': {'m': None, 'postprocess': None}}}

In [52]:
# Run experiment (depending on experiment_type)

results = []

i = 0
for name, config in model_configs.items():
    for logn in config['logn']:

        for v in varying_variable_values:
            kwargs = deepcopy(config['kwargs'])
            kwargs[varying_variable] = v
            model_name = f"{name}_{v}"
            trials = (1 if name in ['full'] else n_repeats)

            X, y = get_toy_dataset(
                X_name=X_name,
                f_name=f_name,
                n=2**logn,
                X_var=X_var,
                d=kwargs['d'] if 'd' in kwargs else d,
                noise=noise,
                M=kwargs['M'] if 'M' in kwargs else M,
                k=k,
            )
            
            # Set kernel, alpha, sigma params
            if 'kernel' not in kwargs:
                kwargs['kernel'] = kernel

            # if name in ['st', 'kt']:
            #     # NOTE: I think you need to set alpha to be proportional to sqrt(n)
            #     kwargs['alpha'] /= np.power(2**logn, 1/4)
                
            
            model = get_estimator(task, name=name, **kwargs)
            if model is None: continue
            print(f'i={i+1}: logn={logn}, model={model}')

            # STEP 2: Get optimal parameters through grid search
            # NOTE: we do something slightly better than k-fold cross validation.
            # Namely, we are trying to get rid of randomness in the Kernel Thinning (or Standard Thinning) routine,
            # but if we did 100-fold CV, then the validation set would be 1% of the data
            # (which is too small to get a good estimate of the validation score).
            # Instead we use the same train-val split for each parameter setting and repeat `trials` times
            if use_cross_validation:
                X_concat, y_concat = np.concatenate([X, X_val]), np.concatenate([y, y_val])
                split = [(np.arange(len(X)), np.arange(len(X), len(X)+len(X_val))) for _ in range(trials)]
                grid_search = GridSearchCV(
                    estimator=model,
                    param_grid=config['param_grid'],
                    return_train_score=True,
                    cv=split,
                    scoring=refit,
                    refit=False,
                    n_jobs=n_jobs,
                ).fit(X_concat, y_concat)
                # get validation scores
                cv_results = pd.DataFrame(grid_search.cv_results_)
                val_scores = []
                for i in range(trials):
                    val_scores.append( cv_results.iloc[grid_search.best_index_][f'split{i}_test_score'] )

                # get optimal parameters
                best_params = grid_search.best_params_
            else:
                # Dummy values
                val_scores = [1,] * trials
                
                best_params = {
                    'sigma' : sigma,
                    'alpha' : alpha, # * (len(X_train)**(1/4) if name in ['st', 'kt'] else 1),
                }    

            print(f"best params: {best_params}")            
            best_model = get_estimator(task, name=name, 
                                       sigma=best_params['sigma'],
                                       alpha=best_params['alpha'],
                                       **kwargs)

            mean_scores = []
            for _ in range(trials):
                best_model.fit(X, y)

                # compute test score
                test_pred = best_model.predict(X_test).squeeze()

                if refit == 'neg_mean_squared_error':
                    test_scores = mean_squared_error(y_test, test_pred)
                elif refit == 'accuracy':
                    test_scores = accuracy_score(y_test, test_pred)
                    # test_scores = [accuracy_score([y], [pred]) for y, pred in zip(y_test, test_pred)]
                else:
                    raise ValueError(f"invalid refit metric: {refit}")

                mean_scores.append( np.mean(test_scores) )
                # std_scores.append( np.std(test_scores) / np.sqrt(len(test_scores)-1) ) # biased estimator of std

            results.append({
                "logn": logn, 
                "model": model_name, 
                "cv_results": pd.DataFrame(grid_search.cv_results_) if use_cross_validation else None,
                "best_index_" : grid_search.best_index_ if use_cross_validation else 0,
                "mean_scores" : mean_scores,
                # "std_score" : np.std(mean_scores),
            })

            i += 1

sampling dataset with params ToyData(X_name=unif, f_name=sum-gauss, X_var=1, d=1, noise=0.1, M=2, k=4)
distances: (256, 4)
i=1: logn=8, model=KernelRidgeRegressor(kernel='gauss')
best params: {'sigma': 0.125, 'alpha': 0.0001}
distances: (256, 256)
distances: (256, 10000)
i=2: logn=8, model=KernelRidgeRegressor()
best params: {'sigma': 0.125, 'alpha': 0.0001}
distances: (256, 256)
distances: (256, 10000)
sampling dataset with params ToyData(X_name=unif, f_name=sum-gauss, X_var=1, d=1, noise=0.1, M=2, k=4)
distances: (1024, 4)
i=3: logn=10, model=KernelRidgeRegressor(kernel='gauss')
best params: {'sigma': 0.125, 'alpha': 0.0001}
distances: (1024, 1024)
distances: (1024, 10000)
i=4: logn=10, model=KernelRidgeRegressor()
best params: {'sigma': 0.125, 'alpha': 0.0001}
distances: (1024, 1024)
distances: (1024, 10000)
sampling dataset with params ToyData(X_name=unif, f_name=sum-gauss, X_var=1, d=1, noise=0.1, M=2, k=4)
distances: (4096, 4)
i=5: logn=12, model=KernelRidgeRegressor(kernel='gaus

In [63]:
results

[{'logn': 8,
  'model': 'full_gauss',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.011906948714003157]},
 {'logn': 8,
  'model': 'full_laplace',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.018287961340612734]},
 {'logn': 10,
  'model': 'full_gauss',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.01019523450940385]},
 {'logn': 10,
  'model': 'full_laplace',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.016187866466939073]},
 {'logn': 12,
  'model': 'full_gauss',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.009960590914306065]},
 {'logn': 12,
  'model': 'full_laplace',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.016465442249291685]},
 {'logn': 14,
  'model': 'full_gauss',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.009851303747097412]},
 {'logn': 14,
  'model': 'full_laplace',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.01650149374506168]},
 {'logn': 8,

In [53]:
# Save results with pickle
if save:
    import pickle

    pickle_file = filename + '.p'
    print(pickle_file)

    with open(pickle_file, 'wb') as f:
        pickle.dump(results, f)

## Plot Results

In [54]:
from functools import reduce
from operator import concat

### Test scores

In [55]:
row_subplot_titles = ["Test Score vs n", "Log2 Test Score vs n"] #, "Train time vs n", "Predict time vs n"]

fig = make_subplots(
    rows=len(row_subplot_titles),
    cols=len(varying_variable_values),
    shared_yaxes=True,
    subplot_titles=reduce(concat, [[f'{varying_variable}={v}' for v in varying_variable_values] for _ in row_subplot_titles]),
    vertical_spacing=0.1,
)

model_names = list(model_configs.keys())
colors_list = colors.qualitative.Plotly * (
    len(model_names) // len(colors.qualitative.Plotly) + 1
)
colors_used = set()

In [56]:
def plot_vs_n(print_name, attr_name, vvv, r, c, is_better='higher', scale='log2'):
    """
    Args:
    - vvv: varying variable value
    """
    
    for result in results:
        model_name = result["model"]
        model_name_prefix, vv_name = model_name.split('_') # E.g., kt_rbf -> (kt, rbf)

        # only select results with the correct varying variable value
        if vv_name != vvv:
            continue

        color = colors_list[model_names.index(model_name_prefix)]

        if scale == 'log2':
            y = np.log2(np.abs(result[f"mean_scores"]))
            hline = np.log2(np.abs(baseline_loss))

        elif scale == 'linear':
            hline = np.abs(baseline_loss)
            y = np.abs(result[f"mean_scores"])

        trace = go.Box(
            x=[result['logn']]*len(result[f"mean_scores"]),
            y=y,
            name=model_name_prefix,
            # opacity=0.5,
            legendgroup=model_name_prefix,
            line_color=color,
            offsetgroup=model_name_prefix,
            showlegend=color not in colors_used,
            boxmean=True,
        )

        fig.add_trace(trace, row=r, col=c)
        colors_used.add(color)

    # add line for baseline loss
    fig.add_hline(
        y=hline,
        row=r, col=c, line_dash="dash",
    )

    if c == 1: fig.update_yaxes(title_text=f"{scale}({print_name}) - {is_better} is better", row=r, col=c)
    fig.update_xaxes(title_text="log2(n)", type='linear', row=r, col=c)
    fig.update_yaxes(type='linear', row=r, col=c)
    fig.update_layout(boxmode='group')

def plot_test_score_vs_n(vvv, r, c, scale):
    plot_vs_n(f"Test MSE", "score", vvv, r, c, is_better='lower', scale=scale)

# def plot_val_score_vs_n(vvv, r, c):
#     plot_vs_n("Val MSE score", "test_score", vvv, r, c, is_better='lower')

# def plot_train_time_vs_n(vvv, r, c):
#     plot_vs_n("Train time", "fit_time", vvv, r, c, is_better='lower')

# def plot_test_time_vs_n(vvv, r, c):
#     plot_vs_n("Test time", "score_time", vvv, r, c, is_better='lower')

In [57]:
for c, vvv in enumerate(varying_variable_values):
    plot_test_score_vs_n(str(vvv), 1, c+1, scale='linear')
    plot_test_score_vs_n(str(vvv), 2, c+1, scale='log2')
    # plot_val_score_vs_n(str(vvv), 2, c+1)
    
    # plot_train_time_vs_n(str(vvv), 3, c+1)
    # plot_test_time_vs_n(str(vvv), 4, c+1)


In [58]:
fig.update_layout(
    # legend=dict(traceorder="normal", borderwidth=1),
    width=800,
    height=800,
    showlegend=True,
    title=f'f={f_name}(k={k}, noise={noise})',
)

In [59]:
if save:
    fig_file = 'figures/' + filename + '_results.png'
    print(fig_file)
    fig.write_image(fig_file)

### Excess risk trendline

In [60]:
trendline_data = {vv_name : {name : {'x':[], 'y':[], 'y_std':[]} for name in model_names} for vv_name in varying_variable_values}
for result in results:
    model_name = result["model"]
    model_name_prefix, vv_name = model_name.split('_') # E.g., kt_rbf -> (kt, rbf)
    
    excess_risk = np.maximum(np.abs(result[f"mean_scores"]) - np.abs(baseline_loss), 0)
    log_excess_risk = np.log2(excess_risk)

    trendline_data[vv_name][model_name_prefix]['x'].append(result['logn'])
    # to avoid errors with taking log of negative numbers
    trendline_data[vv_name][model_name_prefix]['y'].append( np.mean(log_excess_risk) )
    trendline_data[vv_name][model_name_prefix]['y_std'].append(np.std(log_excess_risk))

/var/folders/g7/tv_m7tdj25q_nq7w910g22qw0000gp/T/ipykernel_70991/327199001.py:7: RuntimeWarning:

invalid value encountered in log2



In [61]:
fig = make_subplots(
    rows=1, cols=len(varying_variable_values),
    subplot_titles=varying_variable_values,
    shared_yaxes=True,
)

for c, vv_name in enumerate(varying_variable_values):
    for model_name in model_names:
        x = trendline_data[vv_name][model_name]['x']
        y = trendline_data[vv_name][model_name]['y']

        if len(x) == 0: continue

        # add scatter
        scatter = go.Scatter(
            x=x,
            y=y,
            name=model_name,
            legendgroup=model_name,
            error_y=dict(
                type='data',
                array=trendline_data[vv_name][model_name]['y_std'],
                visible=True,
            ),
            # use markers
            mode='markers',
            marker=dict(
                color=colors_list[model_names.index(model_name)],
            )
        )
        fig.add_trace(scatter, row=1, col=c+1)

        # add trendline
        try:
            z = np.polyfit(x,y,1)
            y_hat = np.poly1d(z)(x)
            fig.add_trace(go.Scatter(
                x=x,
                y=y_hat,
                name=model_name,
                legendgroup=model_name,
                showlegend=False,
                line=dict(
                    dash='dash',
                    color=scatter['marker']['color'],
                ),
            ), row=1, col=c+1)

            # add slope annotation
            fig.add_annotation(
                xref="paper", yref="paper",
                x=min(x), y=model_names.index(model_name),
                text=f"{model_name}: n^{z[0]:.2f}",
                showarrow=False,
                row=1, col=c+1,
            )
        except LinAlgError:
            print(f"cannot fit trendline for {model_name} with varying variable {vv_name}")

fig.update_layout(
    width=800,
    height=400,
    title=f'log(Excess MSE) vs n: f={f_name}(k={k}, noise={noise})',
)
# yaxis title
fig.update_yaxes(title_text=f'log2(Excess MSE)', row=1, col=1)
# xaxis title
fig.update_xaxes(title_text='log2(n)', row=1, col=1)
fig.update_xaxes(title_text='log2(n)', row=1, col=2)
fig.show()

In [62]:
if save:
    fig_file = 'figures/' + filename + '_trendline.png'
    print(fig_file)
    fig.write_image(fig_file)